In [1]:
# Install required libraries
!pip install transformers datasets accelerate evaluate -q
print("Libraries installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.5 MB/s eta 0:00:00
Libraries installed successfully!


In [2]:
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

Device: cuda
GPU: Tesla T4
Memory: 15.83 GB


In [3]:
# Load Grammarly CoEdIT dataset
print("Loading dataset...")
dataset = load_dataset("grammarly/coedit")

print("\nDataset structure:")
print(dataset)

Loading dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.jsonl:   0%|          | 0.00/19.7M [00:00<?, ?B/s]

validation.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/69071 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1712 [00:00<?, ? examples/s]


Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['_id', 'task', 'src', 'tgt'],
        num_rows: 69071
    })
    validation: Dataset({
        features: ['_id', 'task', 'src', 'tgt'],
        num_rows: 1712
    })
})


In [5]:
# Filter for grammar/fluency improvement tasks
def is_grammar_task(example):
    """Keep examples related to grammar correction and text improvement"""
    grammar_keywords = ['grammar', 'fluency', 'clarity', 'coherence', 'fix', 'correct']
    task = example.get('task', '').lower()
    return any(keyword in task for keyword in grammar_keywords)

filtered_dataset = dataset.filter(is_grammar_task)

print("Dataset size after filtering:")
print(f"Train: {len(filtered_dataset['train'])}")
print(f"Validation: {len(filtered_dataset['validation'])}")

Filter:   0%|          | 0/69071 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1712 [00:00<?, ? examples/s]

Dataset size after filtering:
Train: 11868
Validation: 0


In [7]:
# Split validation set to create test set
train_val_split = filtered_dataset['train'].train_test_split(test_size=0.2, seed=42)
val_test_split = train_val_split['test'].train_test_split(test_size=0.5, seed=42)

final_dataset = DatasetDict({
    'train': train_val_split['train'],
    'validation': val_test_split['train'],
    'test': val_test_split['test']
})

print("Final dataset split:")
print(f"Train:      {len(final_dataset['train'])} examples (80%)")
print(f"Validation: {len(final_dataset['validation'])} examples (10%)")
print(f"Test:       {len(final_dataset['test'])} examples (10%)")

Final dataset split:
Train:      9494 examples (80%)
Validation: 1187 examples (10%)
Test:       1187 examples (10%)


In [8]:
# Load FLAN-T5 tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(f"Tokenizer loaded: {model_name}")
print(f"Vocab size: {len(tokenizer)}")

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Tokenizer loaded: google/flan-t5-base
Vocab size: 32100


In [9]:
# Define preprocessing function
def preprocess_function(examples):
    """
    Tokenize input and target texts for FLAN-T5
    Format: "grammar: <input_text>" -> "<corrected_text>"
    """
    # Add task prefix to input
    inputs = ["grammar: " + text for text in examples['src']]
    targets = examples['tgt']

    # Tokenize inputs
    model_inputs = tokenizer(
        inputs,
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    # Tokenize targets
    labels = tokenizer(
        targets,
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

print("Preprocessing function defined")

Preprocessing function defined


In [10]:
# Apply preprocessing to all splits
print("Tokenizing dataset...")
tokenized_dataset = final_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=final_dataset['train'].column_names
)

print("\nTokenization complete!")
print(f"Train samples: {len(tokenized_dataset['train'])}")
print(f"Sample tokenized input shape: {len(tokenized_dataset['train'][0]['input_ids'])}")

Tokenizing dataset...


Map:   0%|          | 0/9494 [00:00<?, ? examples/s]

Map:   0%|          | 0/1187 [00:00<?, ? examples/s]

Map:   0%|          | 0/1187 [00:00<?, ? examples/s]


Tokenization complete!
Train samples: 9494
Sample tokenized input shape: 128


In [29]:
# Check what kind of grammar errors the model actually learned
print("What does our training data look like?\n")

for i in range(10):
    example = final_dataset['train'][i]
    print(f"Example {i+1}:")
    print(f"  Input:  {example['src']}")
    print(f"  Output: {example['tgt']}")
    print("-" * 80)

What does our training data look like?

Example 1:
  Input:  Make the text more cohesive: The owner of the nearby beach house where Lucia, Hanon, Rina, and Kaito work part-time. Kaito is a Kaito and still keeps a picture of Kaito's late wife Saori at his bar.
  Output: The owner of the nearby beach house where Lucia, Hanon, Rina, and Kaito work part-time. He is a widower and still keeps a picture of his late wife Saori at his bar.
--------------------------------------------------------------------------------
Example 2:
  Input:  Fix coherence in the text: Suffering severe losses. He rallied his remaining ships and rescued several of his ships ; most importantly, the grain convoy reached Brest unmolested.
  Output: Although suffering severe losses, he rallied his remaining ships and rescued several of his ships ; most importantly, the grain convoy reached Brest unmolested.
--------------------------------------------------------------------------------
Example 3:
  Input:  Fix coheren

In [11]:
# Load pre-trained Flan-t5 model
print("Loading model...")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Model loaded: {model_name}")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model size: ~{total_params * 4 / 1e9:.2f} GB")

Loading model...


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Model loaded: google/flan-t5-base
Total parameters: 247,577,856
Trainable parameters: 247,577,856
Model size: ~0.99 GB


In [23]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-grammar-v1",

    # Training hyperparameters
    num_train_epochs=3,
    per_device_train_batch_size=4,  # Reduced batch size
    per_device_eval_batch_size=4,
    learning_rate=3e-4,  # Higher learning rate for T5
    weight_decay=0.01,
    warmup_steps=500,

    # Evaluation and saving
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="loss",

    # Logging
    logging_steps=50,
    report_to="none",

    # NO fp16 to avoid nan
    fp16=False,
    predict_with_generate=False,
)

print("Training configuration (Config 1 - Fixed):")
print(f"Epochs: {training_args.num_train_epochs}")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Learning rate: {training_args.learning_rate}")
print(f"FP16: {training_args.fp16}")

Training configuration (Config 1 - Fixed):
Epochs: 3
Batch size: 4
Learning rate: 0.0003
FP16: False


In [24]:
# Verify tokenized data is correct
sample = tokenized_dataset['train'][0]
print("Sample tokenized data:")
print(f"Input IDs shape: {len(sample['input_ids'])}")
print(f"Labels shape: {len(sample['labels'])}")
print(f"Input IDs (first 10): {sample['input_ids'][:10]}")
print(f"Labels (first 10): {sample['labels'][:10]}")

Sample tokenized data:
Input IDs shape: 128
Labels shape: 128
Input IDs (first 10): [19519, 10, 1796, 8, 1499, 72, 29137, 10, 37, 2527]
Labels (first 10): [37, 2527, 13, 8, 4676, 2608, 629, 213, 11977, 9]


In [25]:
from transformers import Seq2SeqTrainer, DataCollatorForSeq2Seq

# Reload model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.to(device)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Initialize Trainer WITHOUT compute_metrics
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
)

print("Trainer initialized (using loss-based evaluation)")

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Trainer initialized (using loss-based evaluation)


In [26]:
print("=" * 60)
print("Starting training - Configuration 1")
print("=" * 60)

trainer.train()

print("\n✓ Training completed!")

Starting training - Configuration 1


Step,Training Loss,Validation Loss
500,2.302228,1.647920
1000,0.685906,0.452831
1500,0.404823,0.260995
2000,0.281603,0.179231
2500,0.188516,0.141273
3000,0.152938,0.119941
3500,0.136126,0.106589
4000,0.120788,0.099429
4500,0.116512,0.093466
5000,0.101391,0.089624


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].



✓ Training completed!


In [27]:
# Evaluate on test set
print("Evaluating on test set...")
test_results = trainer.evaluate(tokenized_dataset["test"])

print("\nTest Set Results:")
print(f"Test Loss: {test_results['eval_loss']:.4f}")

Evaluating on test set...



Test Set Results:
Test Loss: 0.0784


In [30]:
# Corrected test function - no extra prefix needed
def test_coherence_improvement(text):
    """Test the fine-tuned model on coherence tasks"""
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True).to(device)

    outputs = model.generate(
        **inputs,
        max_length=128,
        num_beams=4,
        early_stopping=True
    )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

# Test with examples matching training data style
coherence_test_cases = [
    "Fix coherence in the text: The weather was nice. We stayed home all day.",
    "Make the text more cohesive: She studied hard. She failed the exam.",
    "Improve the consistency of the text: The restaurant opened in 1990. It serves Italian food. It won many awards.",
    "Fix coherence in this sentence: He was tired. He kept working on the project.",
    "Make the text more coherent: The company grew rapidly. The CEO resigned suddenly.",
]

print("Testing Coherence Improvement Model:\n")
for i, test_text in enumerate(coherence_test_cases, 1):
    result = test_coherence_improvement(test_text)
    print(f"Example {i}:")
    print(f"  Input:  {test_text}")
    print(f"  Output: {result}")
    print()

Testing Coherence Improvement Model:

Example 1:
  Input:  Fix coherence in the text: The weather was nice. We stayed home all day.
  Output: The weather was good, but we stayed home all day.

Example 2:
  Input:  Make the text more cohesive: She studied hard. She failed the exam.
  Output: Although she studied hard, she failed the exam.

Example 3:
  Input:  Improve the consistency of the text: The restaurant opened in 1990. It serves Italian food. It won many awards.
  Output: The restaurant opened in 1990 and it serves Italian food. It won many awards.

Example 4:
  Input:  Fix coherence in this sentence: He was tired. He kept working on the project.
  Output: Although he was tired, he kept working on the project.

Example 5:
  Input:  Make the text more coherent: The company grew rapidly. The CEO resigned suddenly.
  Output: The company grew rapidly, but the CEO resigned suddenly.



In [31]:
# Save Config 1 results for comparison
config1_results = {
    'name': 'Config 1: Baseline',
    'learning_rate': 3e-4,
    'batch_size': 4,
    'epochs': 3,
    'final_train_loss': 0.0840,
    'final_val_loss': 0.0822,
    'test_loss': 0.0784
}

print("Config 1 results saved")

Config 1 results saved


In [32]:
# Configuration 2: Lower learning rate
training_args_v2 = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-grammar-v2",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=1e-4,  # LOWER learning rate
    weight_decay=0.01,
    warmup_steps=500,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    logging_steps=50,
    report_to="none",
    fp16=False,
)

print("Config 2: Lower learning rate (1e-4)")

Config 2: Lower learning rate (1e-4)


In [33]:
# Reload fresh model for Config 2
model_v2 = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
data_collator_v2 = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_v2)

trainer_v2 = Seq2SeqTrainer(
    model=model_v2,
    args=training_args_v2,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator_v2,
)

print("="*60)
print("Training Config 2 - Lower LR")
print("="*60)
trainer_v2.train()
print("\n✓ Config 2 training completed!")

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Training Config 2 - Lower LR


Step,Training Loss,Validation Loss
500,4.389949,3.854547
1000,2.247561,1.781496
1500,1.169648,0.894661
2000,0.885138,0.646924
2500,0.680682,0.517805
3000,0.596579,0.435599
3500,0.532896,0.374329
4000,0.478331,0.329745
4500,0.433046,0.296231
5000,0.400985,0.271359


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].



✓ Config 2 training completed!


In [34]:
# Evaluate Config 2 on test set
test_results_v2 = trainer_v2.evaluate(tokenized_dataset["test"])

config2_results = {
    'name': 'Config 2: Lower LR',
    'learning_rate': 1e-4,
    'batch_size': 4,
    'epochs': 3,
    'final_train_loss': 0.3464,
    'final_val_loss': 0.2295,
    'test_loss': test_results_v2['eval_loss']
}

print(f"Config 2 Test Loss: {config2_results['test_loss']:.4f}")

Config 2 Test Loss: 0.2292


In [4]:
# Define all 3 configuration results
config1_results = {
    'name': 'Config 1: Baseline',
    'learning_rate': 3e-4,
    'batch_size': 4,
    'epochs': 3,
    'final_train_loss': 0.0840,
    'final_val_loss': 0.0822,
    'test_loss': 0.0784
}

config2_results = {
    'name': 'Config 2: Lower LR',
    'learning_rate': 1e-4,
    'batch_size': 4,
    'epochs': 3,
    'final_train_loss': 0.3464,
    'final_val_loss': 0.2295,
    'test_loss': 0.2292
}

# Config 3 was trained on Kaggle due to Colab GPU limits
# Results copied from Kaggle training
config3_results = {
    'name': 'Config 3: Fewer Epochs',
    'learning_rate': 3e-4,
    'batch_size': 4,
    'epochs': 2,
    'final_train_loss': 0.0412,
    'final_val_loss': 0.0504,
    'test_loss': 0.0476
}

print("All 3 configurations defined")

All 3 configurations defined


In [5]:
import pandas as pd

# Create comparison table
comparison = pd.DataFrame([config1_results, config2_results, config3_results])

print("\nHyperparameter Optimization Results:\n")
print(comparison.to_string(index=False))

print("\n" + "="*60)
print(f"Best Configuration: {comparison.loc[comparison['test_loss'].idxmin(), 'name']}")
print(f"Best Test Loss: {comparison['test_loss'].min():.4f}")


Hyperparameter Optimization Results:

                  name  learning_rate  batch_size  epochs  final_train_loss  final_val_loss  test_loss
    Config 1: Baseline         0.0003           4       3            0.0840          0.0822     0.0784
    Config 2: Lower LR         0.0001           4       3            0.3464          0.2295     0.2292
Config 3: Fewer Epochs         0.0003           4       2            0.0412          0.0504     0.0476

Best Configuration: Config 3: Fewer Epochs
Best Test Loss: 0.0476


In [10]:
# Reload dataset and tokenize (needed if starting fresh session)
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch, random

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Reload and filter dataset
dataset = load_dataset("grammarly/coedit")

def is_grammar_task(example):
    grammar_keywords = ['grammar', 'fluency', 'clarity', 'coherence', 'fix', 'correct']
    return any(k in example.get('task', '').lower() for k in grammar_keywords)

filtered = dataset.filter(is_grammar_task)
split1 = filtered['train'].train_test_split(test_size=0.2, seed=42)
split2 = split1['test'].train_test_split(test_size=0.5, seed=42)
final_dataset = DatasetDict({'train': split1['train'], 'validation': split2['train'], 'test': split2['test']})

def preprocess_function(examples):
    inputs = ["grammar: " + t for t in examples['src']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(examples['tgt'], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = final_dataset.map(preprocess_function, batched=True, remove_columns=final_dataset['train'].column_names)
print(f"Test set ready: {len(tokenized_dataset['test'])} samples")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

Filter:   0%|          | 0/1712 [00:00<?, ? examples/s]

Map:   0%|          | 0/9494 [00:00<?, ? examples/s]

Map:   0%|          | 0/1187 [00:00<?, ? examples/s]

Map:   0%|          | 0/1187 [00:00<?, ? examples/s]

Test set ready: 1187 samples


In [11]:
# Baseline comparison - evaluate original FLAN-T5
from transformers import AutoModelForSeq2SeqLM
import random

print("Evaluating baseline (original FLAN-T5) model...")

# Load original model (not fine-tuned)
baseline_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
baseline_model.eval()

# Evaluate on subset of test set (100 samples)
def evaluate_baseline_loss(model, dataset, num_samples=100):
    """Calculate average loss on test samples"""
    indices = random.sample(range(len(dataset)), num_samples)
    total_loss = 0

    with torch.no_grad():
        for idx in indices:
            sample = dataset[idx]
            inputs = torch.tensor([sample['input_ids']])
            labels = torch.tensor([sample['labels']])

            # Replace -100 with pad_token_id for baseline
            labels_clean = labels.clone()
            labels_clean[labels_clean == -100] = 0

            outputs = model(input_ids=inputs, labels=labels_clean)
            total_loss += outputs.loss.item()

    return total_loss / num_samples

baseline_loss = evaluate_baseline_loss(baseline_model, tokenized_dataset['test'], 100)

print(f"\nBaseline Test Loss: {baseline_loss:.4f}")
print(f"Fine-tuned (Config 3) Test Loss: {config3_results['test_loss']:.4f}")
print(f"Improvement: {((baseline_loss - config3_results['test_loss']) / baseline_loss * 100):.1f}%")

Evaluating baseline (original FLAN-T5) model...


Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning



Baseline Test Loss: 11.0116
Fine-tuned (Config 3) Test Loss: 0.0476
Improvement: 99.6%


In [13]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "google/flan-t5-base"

# Load model
print("Loading model...")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Training config (Config 3 - best)
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-grammar-best",
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=3e-4,
    weight_decay=0.01,
    warmup_steps=500,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    logging_steps=100,
    report_to="none",
    fp16=False,
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
)

print("Starting training (Config 3 - best)...")
trainer.train()

# Save final model
model.save_pretrained("./flan-t5-grammar-final")
tokenizer.save_pretrained("./flan-t5-grammar-final")
print("\nModel saved to ./flan-t5-grammar-final")

Loading model...


Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Starting training (Config 3 - best)...


Step,Training Loss,Validation Loss
500,2.543621,1.647920
1000,0.707640,0.456017
1500,0.440579,0.267084
2000,0.317830,0.187933
2500,0.211141,0.152553
3000,0.181996,0.129010
3500,0.164826,0.118559
4000,0.146764,0.111283
4500,0.138808,0.107381


...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]


Model saved to ./flan-t5-grammar-final


In [14]:
# Error Analysis
import random

model.eval()

# Get 30 random test examples
random.seed(42)
test_indices = random.sample(range(len(final_dataset['test'])), 30)

results = []
for idx in test_indices:
    example = final_dataset['test'][idx]
    input_text = example['src']
    expected = example['tgt']

    inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    predicted = tokenizer.decode(outputs[0], skip_special_tokens=True)

    match = "MATCH" if predicted.strip() == expected.strip() else "MISMATCH"
    results.append({
        'input': input_text,
        'expected': expected,
        'predicted': predicted,
        'match': match
    })

# Summary
matches = sum(1 for r in results if r['match'] == 'MATCH')
print(f"Exact Match: {matches}/{len(results)} ({matches/len(results)*100:.1f}%)\n")

# Show mismatches for analysis
print("=" * 70)
print("ERROR ANALYSIS - Mismatch Examples")
print("=" * 70)
mismatch_count = 0
for r in results:
    if r['match'] == 'MISMATCH':
        mismatch_count += 1
        if mismatch_count <= 10:
            print(f"\nMismatch {mismatch_count}:")
            print(f"  Input:    {r['input'][:120]}")
            print(f"  Expected: {r['expected'][:120]}")
            print(f"  Got:      {r['predicted'][:120]}")
            print("-" * 70)

print(f"\nTotal mismatches: {mismatch_count}/{len(results)}")

# Show successful matches
print("\n" + "=" * 70)
print("SUCCESSFUL CORRECTIONS")
print("=" * 70)
match_count = 0
for r in results:
    if r['match'] == 'MATCH':
        match_count += 1
        if match_count <= 5:
            print(f"\nSuccess {match_count}:")
            print(f"  Input:  {r['input'][:120]}")
            print(f"  Output: {r['predicted'][:120]}")
            print("-" * 70)

Exact Match: 8/30 (26.7%)

ERROR ANALYSIS - Mismatch Examples

Mismatch 1:
  Input:    Clarify this paragraph: Many online comments on social media platforms are hateful, humorous, or sarcastic.
  Expected: Sentiment analysis of social media comments is very important for review analysis. Many online reviews are sarcastic, hu
  Got:      Many online comments on social media platforms are hateful, humorous, or sarcastic.
----------------------------------------------------------------------

Mismatch 2:
  Input:    Make this sentence better readable: As the crisis escalated, was a high demand for contact tracers, and the CDC had earl
  Expected: As the crisis escalated, there was a high demand for contact tracers, and the CDC had earlier named librarians as key pu
  Got:      As the crisis escalated, was a high demand for contact tracers, and the CDC had earlier named librarians as key public h
----------------------------------------------------------------------

Mismatch 3:
  Input: 

In [16]:
# Error Analysis Summary
print("ERROR CATEGORIES IDENTIFIED:\n")
print("Category A - Different but Valid Corrections (most common):")
print("  Model uses different connective words (e.g., 'However' vs 'For example')")
print("  Both versions are grammatically correct and coherent\n")
print("Category B - Minor Word Choice Differences:")
print("  Model picks slightly different words but preserves meaning\n")
print("Category C - Incomplete Corrections:")
print("  Model fails to add missing words in some cases\n")
print("Category D - Over-conservative (rare):")
print("  Model returns input unchanged when major rewriting is needed\n")
print("IMPROVEMENT SUGGESTIONS:")
print("  - Use BLEU/ROUGE metrics instead of exact match for fairer evaluation")
print("  - Train with more diverse paragraph-level examples")
print("  - Increase max_length for longer text corrections")

# Inference Pipeline
def improve_text(text, task="Fix coherence in the text"):
    input_text = f"{task}: {text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\n" + "=" * 70)
print("Inference Pipeline Demo")
print("=" * 70 + "\n")

demo_inputs = [
    ("The weather was nice. We stayed home.", "Fix coherence in the text"),
    ("She studied hard. She failed the exam.", "Make the text more cohesive"),
    ("He was tired. He kept working.", "Fix coherence in this sentence"),
    ("The company grew fast. The CEO quit.", "Make the text more coherent"),
    ("I like pizza. I eat it every day. It is cheap.", "Improve the consistency of the text"),
]

for i, (text, task) in enumerate(demo_inputs, 1):
    result = improve_text(text, task)
    print(f"Example {i}:")
    print(f"  Input:  {text}")
    print(f"  Output: {result}\n")

ERROR CATEGORIES IDENTIFIED:

Category A - Different but Valid Corrections (most common):
  Model uses different connective words (e.g., 'However' vs 'For example')
  Both versions are grammatically correct and coherent

Category B - Minor Word Choice Differences:
  Model picks slightly different words but preserves meaning

Category C - Incomplete Corrections:
  Model fails to add missing words in some cases

Category D - Over-conservative (rare):
  Model returns input unchanged when major rewriting is needed

IMPROVEMENT SUGGESTIONS:
  - Use BLEU/ROUGE metrics instead of exact match for fairer evaluation
  - Train with more diverse paragraph-level examples
  - Increase max_length for longer text corrections

Inference Pipeline Demo

Example 1:
  Input:  The weather was nice. We stayed home.
  Output: The weather was good, but we stayed home.

Example 2:
  Input:  She studied hard. She failed the exam.
  Output: Studying hard, she failed the exam.

Example 3:
  Input:  He was tired. H